In [1]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np
import statistics 

wine = datasets.load_wine()
print('features: ' + str(wine.feature_names) + '\n')
print('targets:  ' + str(wine.target_names) + '\n')

X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.3) # 70% training and 30% test
t_samples, t_features = X_train.shape
t_labels = wine.target_names
num_labels = len(t_labels)
root=None

features: ['alcohol', 'malic_acid', 'ash', 'alcalinity_of_ash', 'magnesium', 'total_phenols', 'flavanoids', 'nonflavanoid_phenols', 'proanthocyanins', 'color_intensity', 'hue', 'od280/od315_of_diluted_wines', 'proline']

targets:  ['class_0' 'class_1' 'class_2']



In [2]:
#node class
class Node:
    def __init__(self, feature=None, threshhold=None, left=None, right=None, value=None):
        self.feature = feature
        self.thresh = threshhold
        self.left=left
        self.right=right
        self.value=value

    def isLeaf(self):
        if self.value != None: return True
        else :
            return False
        
def fit_tree(X, y):
    root = build_tree(X,y)
    return root

def build_tree(X,y, depth=0):
    num_samples, num_features = X.shape
    num_labels = len(np.unique(y))

    #stopping criteria
    if(depth >= max_depth or num_samples<min_samples or num_labels==1):
        leaf = statistics.mode(y)
        return Node(value=leaf)

    #find best split
    best_feat, best_thresh = best_split(X, y, num_features)

    #create child nodes and recurse
    l_idxs, r_idxs = split(X[:,best_feat], best_thresh)
    left = build_tree(X[l_idxs,:], y[l_idxs], depth+1)
    right = build_tree(X[r_idxs,:], y[r_idxs], depth+1)
    return Node(feature=best_feat,threshhold=best_thresh,left=left,right=right)

#find split with highest info gain
def best_split(X, y, num_features):
    best_ig=-1
    s_feat_index=None
    s_thresh=None
    for f in range(num_features):
        x_col = X[:,f]
        threshs = np.unique(x_col)
        for t in threshs:
            ig = info_gain(x_col, y, t)
            if ig > best_ig:
                best_ig=ig
                s_feat_index = f
                s_thresh = t
    return s_feat_index, s_thresh

def info_gain(x_col, y, thresh):
    #parent entropy
    p_entr = entropy(y)

    #create children
    l_idxs, r_idxs = split(x_col, thresh)
    if (len(l_idxs) == 0 or len(r_idxs) == 0): #if left or right side is empty
        return 0
    
    #children entropy
    r_entr = entropy(y[r_idxs])
    l_entr = entropy(y[l_idxs])

    #weighted average * child entropy
    num_y = len(y)
    w_avg_entr = (len(l_idxs)/num_y) * l_entr + (len(r_idxs)/num_y) * r_entr

    #calc info gain
    ig = p_entr - w_avg_entr
    return ig

#-sum(p(X)*log2(p(X))
def entropy(y):
    count = np.bincount(y)
    probability = count/len(y)
    sum = 0
    for p in probability:
        if p > 0:
            sum += p*np.log(p)
    sum *= -1
    return sum

#get left and right split indexes
def split(x_col, thresh):
    left_indexes = np.argwhere(x_col <= thresh).flatten()
    right_indexes = np.argwhere(x_col > thresh).flatten()
    return (left_indexes, right_indexes)

def predict(X_test):
    predictions = []
    for x in X_test:
        p = traverse_tree(x, root)
        predictions.append(p)
    return predictions

def traverse_tree(x, root):
    if (root.isLeaf()):
        return root.value
    
    if (x[root.feature] <= root.thresh):
        return traverse_tree(x, root.left)
    else:
        return traverse_tree(x, root.right)
    
def accuracy(pred, actual):
    return(np.sum(pred==actual)/(actual.__len__()))

In [3]:
max_depth = 5
min_samples = 5

root = fit_tree(X_train, y_train)
pred=predict(X_test)

print('**SCRATCH DECISION TREE**\n')
print('Scratch Test Accuracy: ')
print(accuracy(pred, y_test))
pred=predict(X_train)
print('Scratch Train Accuracy: ')
print(accuracy(pred, y_train))

## **Sklearn Implementation**

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

clf_model = DecisionTreeClassifier(criterion='entropy', max_depth=5, min_samples_leaf=5, random_state=42)   
clf_model.fit(X_train,y_train)

y_predict = clf_model.predict(X_test)
y_predict_train = clf_model.predict(X_train)

print('\n\n**SKLEARN DECISION TREE**\n')
print('Sklearn Test Accuracy:')
print(accuracy_score(y_test,y_predict))
print('Sklearn Train Accuracy:')
print(accuracy_score(y_train,y_predict_train))

**SCRATCH DECISION TREE**

Scratch Test Accuracy: 
0.8703703703703703
Scratch Train Accuracy: 
0.9838709677419355


**SKLEARN DECISION TREE**

Sklearn Test Accuracy:
0.8333333333333334
Sklearn Train Accuracy:
0.9435483870967742
